# EDA for criminal incidence

In [ ]:
# -*- coding: utf-8 -*-
"""Copy of EDA_criminalIncidence.ipynb
Automatically generated by Colaboratory.
Original file is located at
    https://colab.research.google.com/drive/1EQ8ltMWw1aG-QTvdTKpWMuREbjy8WVXK
## Downloading dataset
"""
""" EDA for criminal incidence dataset
This script allows to put in a csv file the longitude and latitude
for a neighborhood center point, by doing a query with the neighborhood's 
name with the Google Maps API.
This script requires that `pandas` be installed within the Python
environment you are running this script in. Also an API KEY for Google Maps is needed.
This file contains the following functions:
    * get_lat_lng_from_google_maps_api - returns the longitude and latide in the center for an area
    * _populate_results_dict - this function populate a dictionary wuth the info downloaded from Google Maps
"""

import pandas as pd

df = pd.read_csv('https://trello-attachments.s3.amazonaws.com/5e7ab7849f172231e1d8b386/5e7d5e0fec43d718240c71c7/cbf8668259455adfbabd3f686e410b41/incidencia_delictiva_jalisco18-19_filter.csv')

municipalities = ["GUADALAJARA", "ZAPOPAN", "SAN PEDRO TLAQUEPAQUE"]
null_values = ["N.D.", "N..D."]

df = df[df["Municipio"].isin(municipalities)]
df = df[~df["Colonia"].isin(null_values)]


# Get geospatiol information latitude and longitude from Google Maps API

In [ ]:
def get_lat_lng_from_google_maps_api(neighborhoods):

    """Gets and puts in a csv file the longitude and latitude
       for a neighborhood center point, by doing a query with the neighborhood's 
       name with the Google Maps API
    Parameters
    ----------
    neighborhoods : list<str>
        An string list with the neighborhood's names
    
    Returns
    -------
    void :
        the file neighborhoods_latlng.csv
    """


    !pip install -U googlemaps

    from googlemaps import Client as GoogleMaps

    # Don't know how to handle these abbreviations: gral, res, cab. mun.
    abbr_extensions = {
        "jards.": "jardines",
        "nte.": "norte",
        "u.": "unidad",
        "hab.": "habitacional",
        "ote.": "oriente",
        "blvd.": "boulevard",
        "circun.": "circunvalacion",
        "secc.": "seccion",
        "pte.": "poniente",
        "hdas.": "haciendas",
        "dr.": "doctor",
        "hda.": "hacienda",
        "fracc.": "fraccionamiento",
        "fco.": "francisco",
        "conj.": "conjunto",
        "harbioledas": "harboledas",
        "rndas.": "residencias",
        "rnda.": "residencia",
        "hards.": "jardines"
    }

    NEIGHBORHOOD_KEY = "colonia"
    LOCATION_LAT_KEY = "location_lat"
    LOCATION_LNG_KEY = "location_lng"
    GEOMETRY_NE_LAT_KEY = "geometry_ne_lat"
    GEOMETRY_NE_LNG_KEY = "geometry_ne_lng"
    GEOMETRY_SW_LAT_KEY = "geometry_sw_lat"
    GEOMETRY_SW_LNG_KEY = "geometry_sw_lng"
    QUERY_KEY = "query"
    STATUS_KEY = "status"

    res = {
        NEIGHBORHOOD_KEY: [],
        LOCATION_LAT_KEY: [],
        LOCATION_LNG_KEY: [],
        GEOMETRY_NE_LAT_KEY: [],
        GEOMETRY_NE_LNG_KEY: [],
        GEOMETRY_SW_LAT_KEY: [],
        GEOMETRY_SW_LNG_KEY: [],
        QUERY_KEY: [],
        STATUS_KEY: [],
    }

    gmaps = GoogleMaps('APIKEY')

    for neighborhood in neighborhoods:
        neighborhood_low = neighborhood.lower()
        neighborhood_to_search = neighborhood_low

        if '.' in neighborhood_low:
            for token in neighborhood_low.split():
                if token in abbr_extensions:
                    neighborhood_to_search = neighborhood_low.replace(token, abbr_extensions[token])
                    break

        query = neighborhood_to_search

        # The following bounds correspond to the Guadalajara Metropolitan Area, which were recovered from this request:
        # https://maps.googleapis.com/maps/api/geocode/json?address=zona+metropolitana+guadalajara&key=APIKEY
        results = gmaps.geocode(
                query,
                bounds={
                    "southwest": (20.3257581, -103.6650327),
                    "northeast": (20.9982375, -103.0809884),
                },
            )

        def _populate_results_dict(
            neighborhood,
            query,
            status,
            location_lat=None,
            location_lng=None,
            geometry_ne_lat=None,
            geometry_ne_lng=None,
            geometry_sw_lat=None,
            geometry_sw_lng=None
        ):

        """Gets the geospatial information for a neighborhood
            Parameters
            ----------
            neighborhood: str
                The neighborhood's name
            query: str
                The neighborhood's name cleaned
            status: str
                The status query
            location_lat: str
                The neighborhood's latitude
            location_lng: str
                The neighborhood's longitude
            geometry_ne_lat: str
                The north east latitude
            geometry_ne_lng: str
                The north east longitude
            geometry_sw_lat: str
                The south weast latitude
            geometry_sw_lng: str
                The north east latitude
            Returns
            -------
            void
                populates res dictionary
        """

            if location_lat is not None:
                location_lat = location_lat

            if location_lng is not None:
                location_lng = location_lng

            if geometry_ne_lat is not None:
                geometry_ne_lat = geometry_ne_lat

            if geometry_ne_lng is not None:
                geometry_ne_lng = geometry_ne_lng

            if geometry_sw_lat is not None:
                geometry_sw_lat = geometry_sw_lat

            if geometry_sw_lng is not None:
                geometry_sw_lng = geometry_sw_lng

            res[NEIGHBORHOOD_KEY].append(neighborhood)
            res[QUERY_KEY].append(query)
            res[STATUS_KEY].append(status)
            res[LOCATION_LAT_KEY].append(location_lat)
            res[LOCATION_LNG_KEY].append(location_lng)
            res[GEOMETRY_NE_LAT_KEY].append(geometry_ne_lat)
            res[GEOMETRY_NE_LNG_KEY].append(geometry_ne_lng)
            res[GEOMETRY_SW_LAT_KEY].append(geometry_sw_lat)
            res[GEOMETRY_SW_LNG_KEY].append(geometry_sw_lng)

        if len(results) == 0:
            _populate_results_dict(neighborhood, query, "ZERO_RESULTS")
            continue

        geometry_ne_lat = ""
        geometry_ne_lng = ""
        geometry_sw_lat = ""
        geometry_sw_lng = ""

        # "Bounds" is the preferred way to get coordinates as they tend to be more precise,
        # if they're not present, "viewport" will be used.
        try:
            geometry_ne_lat = results[0]['geometry']['bounds']['northeast']['lat']
            geometry_ne_lng = results[0]['geometry']['bounds']['northeast']['lng']
            geometry_sw_lat = results[0]['geometry']['bounds']['southwest']['lat']
            geometry_sw_lng = results[0]['geometry']['bounds']['southwest']['lng']
        except KeyError:
            geometry_ne_lat = results[0]['geometry']['viewport']['northeast']['lat']
            geometry_ne_lng = results[0]['geometry']['viewport']['northeast']['lng']
            geometry_sw_lat = results[0]['geometry']['viewport']['southwest']['lat']
            geometry_sw_lng = results[0]['geometry']['viewport']['southwest']['lng']

        location_lat = results[0]['geometry']['location']['lat']
        location_lng = results[0]['geometry']['location']['lng']

        _populate_results_dict(
            neighborhood,
            query,
            "OK",
            location_lat,
            location_lng,
            geometry_ne_lat,
            geometry_ne_lng,
            geometry_sw_lat,
            geometry_sw_lng
        )

    df_latlng = pd.DataFrame(data=res)
    df_latlng.to_csv("neighborhoods_latlng.csv", index=False)